In [7]:
import os
import json
import dask
import boto3
import pandas as pd
from botocore.config import Config
from boto3.dynamodb.conditions import Attr
from hs_service.aws.dynamodb import DynamoDB

BOTO3_CONFIG = Config(retries={"max_attempts": 10, "mode": "adaptive"})
is_control_tower_migrated = os.getenv("CONTROL_TOWER_MIGRATED")

# standard functions to be leveraged across the workbook
def fetch_accounts_from_metadata(exempted_account_ids=[]):
    dynamodb = DynamoDB(f"{os.getenv('HUB_NAME')}-DYN_METADATA")
    filter_expression = (
        Attr("status").eq("Active")
        & Attr("account-type").ne("Hub")
        & Attr("account-type").ne("Security")
    )

    if exempted_account_ids:
        for acc in exempted_account_ids:
            filter_expression = filter_expression.__and__(Attr("account").ne(acc))

    if is_control_tower_migrated:
        filter_expression = filter_expression.__and__(
            Attr("account-type").ne("Unmanaged")
        )

    if not is_control_tower_migrated:
        filter_expression = filter_expression.__and__(
            Attr("account-type").ne("Foundation")
            & Attr("environment-type").ne("Platform")
        )

    return dynamodb.get_all_entries(filter_expression=filter_expression)


def fetch_all_active_regions():
    client = boto3.client("ec2")
    active_regions = client.describe_regions(
        Filters=[
            {"Name": "opt-in-status", "Values": ["opt-in-not-required", "opted-in"]}
        ]
    )
    return [x["RegionName"] for x in active_regions["Regions"]]


@dask.delayed
def assume_return_boto_client(client_name, account_id, account_region):
    sts_client = boto3.client("sts")
    assumed_role_object = sts_client.assume_role(
        RoleArn=f"arn:aws:iam::{account_id}:role/{os.getenv('ASSUME_ROLE_NAME')}",
        RoleSessionName="LogGroupRetention",
    )
    credentials = assumed_role_object["Credentials"]
    return boto3.client(
        client_name,
        aws_access_key_id=credentials["AccessKeyId"],
        aws_secret_access_key=credentials["SecretAccessKey"],
        aws_session_token=credentials["SessionToken"],
        region_name=account_region,
        config=BOTO3_CONFIG,
    )
